In [1]:
import numpy as np
import pandas as pd
import math

## Part 1: Add Columns with Airline Name and Country to Flight Data

In [2]:
flight_i = pd.read_csv('flightlist_20210401_20210430.csv')

In [3]:
#flight_i.head()

#flight_i.shape[0]

flight_i.isna().sum()

callsign              0
number          1506463
icao24               14
registration     433647
typecode         944834
origin           533498
destination      404396
firstseen             0
lastseen              0
day                   0
latitude_1            0
longitude_1           0
altitude_1            0
latitude_2           55
longitude_2          55
altitude_2        18714
dtype: int64

In [4]:
a_c_map = pd.read_excel('Bansard-airlines-codes-IATA-ICAO.xlsx')

In [5]:
a_c_map.head()

,Airline Name,IATA Designator,3-Digit Code,ICAO Designator,Country
0,ABSA Cargo Airline,M3,NaN,TUS,Brazil
1,Adria Airways,JP,165.0,ADR,Slovenia
2,Aegean Airlines,A3,390.0,AEE,Greece
3,Aer Lingus,EI,53.0,EIN,Ireland
4,Aero República,P5,845.0,RBP,Colombia


In [6]:
# Pull first three chars of callsign
first_3 = list(flight_i['callsign'].str[:3])
#test[:5]

In [7]:
countries = []
airlines = []
icao = list(a_c_map['ICAO Designator'])
a_c_country = list(a_c_map['Country'])
airline_names = list(a_c_map['Airline Name'])
for i in range(len(first_3)):
    if first_3[i] in icao:
        found_idx = icao.index(first_3[i])
        countries.append(a_c_country[found_idx])
        airlines.append(airline_names[found_idx])
    else:
        countries.append(0)
        airlines.append(0)

In [8]:
flight_i['country'] = countries
flight_i['airline'] = airlines
flight_i = flight_i[['callsign', 'airline', 'country','number', 'icao24', 'registration', 'typecode', 
                     'origin', 'destination', 'firstseen', 'lastseen', 'day', 'latitude_1',
                     'longitude_1', 'altitude_1', 'latitude_2', 'longitude_2', 'altitude_2']]
flight_i.head()

,callsign,airline,country,number,icao24,registration,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,CSN495,China Southern Airlines,China (People's Republic of),CZ495,780654,B-2081,B77L,KLAX,EHAM,2021-03-31 00:03:41+00:00,2021-04-01 06:53:24+00:00,2021-04-01 00:00:00+00:00,33.936218,-118.415913,0.0,52.304329,4.778595,-91.44
1,HI1050,0,0,NaN,0c4154,NaN,NaN,KFRG,NaN,2021-03-31 00:20:09+00:00,2021-04-01 03:03:45+00:00,2021-04-01 00:00:00+00:00,40.723339,-73.405151,0.0,-34.220969,-55.710239,4785.36
2,ETH3937,Ethiopian Airlines,Ethiopia,NaN,04014c,NaN,NaN,OMSJ,GABS,2021-03-31 01:44:14+00:00,2021-04-01 14:55:41+00:00,2021-04-01 00:00:00+00:00,25.331074,55.517492,0.0,12.523297,-7.965811,449.58
3,ETH728,Ethiopian Airlines,Ethiopia,NaN,040141,NaN,NaN,WSSS,EBBR,2021-03-31 03:51:28+00:00,2021-04-01 04:25:29+00:00,2021-04-01 00:00:00+00:00,1.328415,103.968879,304.8,50.895608,4.508743,-7.62
4,CES7553,China Eastern,China (People's Republic of),NaN,780b34,B-5931,A332,KLAX,LFPG,2021-03-31 04:48:33+00:00,2021-04-01 13:14:53+00:00,2021-04-01 00:00:00+00:00,33.947381,-118.429985,0.0,49.025000,2.529602,45.72


## Part 2: Impute Missing Origin and Destination Airport Names

In [9]:
# Get list of origin/destination airports and list of lists of lat and long
origins = list(flight_i['origin'])
dests = list(flight_i['destination'])
lat1 = list(flight_i['latitude_1'])
lat2 = list(flight_i['latitude_2'])
long1 = list(flight_i['longitude_1'])
long2 = list(flight_i['longitude_2'])
o_locs = list(zip(origins, lat1, long1))
d_locs = list(zip(dests, lat2, long2))
locs = o_locs + d_locs
len(o_locs),len(locs)

(2227362, 4454724)

In [10]:
# Remove NaN airport name entries
airport_tbl = list(filter(lambda a: type(a[0]) != float and ~math.isnan(a[1]) and ~math.isnan(a[2]), locs))
airport_tbl

[('KLAX', 33.93621826171875, -118.4159134845344),
 ('KFRG', 40.72333901615466, -73.4051513671875),
 ('OMSJ', 25.3310743428893, 55.51749175449587),
 ('WSSS', 1.328414658368644, 103.9688794366245),
 ('KLAX', 33.94738148834745, -118.4299850463867),
 ('KLAX', 33.94752114506091, -118.4274101257324),
 ('EYKA', 54.96427917480469, 24.09245210535386),
 ('WSSS', 1.347702026367188, 103.9768438824153),
 ('EDDF', 50.04552841186523, 8.586140682822778),
 ('EHAM', 52.29683051675052, 4.757315499441964),
 ('EDDF', 50.04111099243164, 8.562090020430714),
 ('SBKP', -23.06090791346662, -47.11288452148438),
 ('ZGSZ', 22.61293029785156, 113.8247014825994),
 ('KLAX', 33.93608093261719, -118.4174829599809),
 ('PANC', 61.20661926269531, -150.018310546875),
 ('OA16', 39.15988210904396, -84.815185546875),
 ('KONT', 34.05674743652344, -117.5924994021046),
 ('VHHH', 22.30368921312235, 113.9211018880208),
 ('EHAM', 52.29138390492584, 4.743190220424108),
 ('VIDP', 28.56669616699219, 77.08726149338942),
 ('WSSS', 1.336

In [11]:
# Define key by which to sort entries, in this case airport name
def get_ap(e):
    return str(e[0])

In [12]:
airport_tbl.sort(key = get_ap)

In [13]:
# Take one entry for each airport
# Note: this may be inaccurate if first found loc is not correctly recorded
# Will do for now, may add additional checks 
ap_clean = []
prev = None
for i in range(len(airport_tbl)):
    if airport_tbl[i][0] != prev:
        ap_clean.append(airport_tbl[i])
    prev = airport_tbl[i][0]
    
ap_clean

[('00AK', 60.07678015757415, -151.6845049176897),
 ('00AL', 34.80775451660156, -86.88470334422829),
 ('00AS', 34.88526942366261, -97.8017807006836),
 ('00AZ', 34.22264099121094, -112.0747282067124),
 ('00CL', 39.3262939453125, -121.8331245754076),
 ('00FA', 28.6978718385858, -82.27849623736216),
 ('00FL', 27.26736812268273, -80.94377957857569),
 ('00GA', 33.73265075683594, -84.10790891063454),
 ('00ID', 48.01675415039062, -116.3997344970703),
 ('00IL', 41.97680457163665, -89.65016209801962),
 ('00IS', 39.9283447265625, -89.12274169921875),
 ('00KS', 38.76144150556144, -94.90106201171876),
 ('00KY', 37.458251953125, -84.70856362200794),
 ('00MD', 38.75569152832031, -75.67836595618206),
 ('00MO', 37.20904541015625, -94.45729357130983),
 ('00NC', 36.08412920418432, -78.42485143783244),
 ('00NY', 42.93489074707031, -77.398681640625),
 ('00OH', 41.55825805664062, -84.06432411887431),
 ('00PN', 41.36824035644531, -80.2467041015625),
 ('00SC', 34.02330818822828, -80.30261993408203),
 ('00TN',

In [14]:
# Gets indices of NaNs in a given column
# Returns a list of indices of NaNs
def get_nas(col):
    nas = []
    for i in range(flight_i.shape[0]):
        if pd.isna(flight_i[col][i]):
            nas.append(i)
    return nas
            

In [15]:
# Get all nan origins
o_na = get_nas('origin')

# Get all nan destinations
d_na = get_nas('destination')

In [17]:
%%time  

# Fills nans for given direction with corresponding list (o_na, d_na)
# Returns a list of origins/destinations that could not be filled
def fill_na(direction, list_na):
    not_found = []
    
    for i in range(len(list_na)):
        lat = ''
        long = ''
        if direction == 'origin':
            lat = 'latitude_1'
            long = 'longitude_1'
        elif direction == 'destination':
            lat = 'latitude_2'
            long = 'longitude_2'
        else:
            print('Incorrect Direction Used: Please use origin or destination')
        cur_idx = list_na[i]
        lat_1 = flight_i[lat][cur_idx]
        long_1 = flight_i[long][cur_idx]
        ap_name = ''
        found = 0
        for j in range(len(ap_clean)):
            if abs(ap_clean[j][1] - lat_1) <= 0.5 and abs(ap_clean[j][2] - long_1) <= 0.5:
                ap_name = ap_clean[j][0]
                found = 1
                
                # Replace NaN with name in big table
                flight_i.at[cur_idx, direction] = ap_name
                break
        if found == 0:
            not_found.append(i)
            flight_i.at[cur_idx, direction] = 0
            #print(found, " ", ap_name, " ", lat_1, " ",long_1)
    #print(not_found)
    return not_found

Wall time: 0 ns


In [ ]:
# Run to fill origins:
fill_na('origin', o_na)

In [ ]:
# Run to fill destinations:
fill_na('destination', d_na)

In [ ]:
#(flight_i['origin'] == 0).sum()

## Save to CSV

In [ ]:
flight_i.to_csv('OSN_Apr2021_Clean.csv', index = False)  

In [ ]:
#(flight_i['destination'] == 0).sum()